# Crawl dữ liệu

Định nghĩa một số hàm cần thiết

In [4]:
import bs4
import pandas as pd
# thư viện dể call api
import requests

def getSoup(url):
  """Hàm tải về một trang web từ một url và trả về một cây HTML"""
  # dùng header của postman để không bị từ chối 
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  return bs4.BeautifulSoup(page.text,"html.parser")

## thegioididong.com

Ngày crawl: 22/6/2022

In [5]:
# url của thegioididong
mainURL="https://www.thegioididong.com"
# url api của thegioididong
apiURL="https://www.thegioididong.com/Category/FilterProductBox?c=42&o=9&pi="

def getLinkFromJSONTGDD(url):
  """Hàm call api của thegioididong sau đó trả về link các sản phẩm"""
  # dùng header của postman để không bị từ chối 
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  # gọi api
  page = requests.get(url,headers=headers)
  # lấy ra trường listproducts
  listproducts=page.json()['listproducts']
  soup=bs4.BeautifulSoup(listproducts,"html.parser")
  links = soup.findAll('a', class_='main-contain')
  return [link.get('href') for link in links]


In [6]:
# Chứa tất cả link sản phẩm
allLinks=[]
# lấy ra tất cả các link từ trang 1 đến trang 5
for i in range(5):
  allLinks+=getLinkFromJSONTGDD(apiURL+str(i))

In [7]:
# Số link lấy được
print("Number of link: ",len(allLinks))

Number of link:  97


In [8]:
tgdd=[]

for (i,link) in enumerate(allLinks):
  #try catch để nếu xảy ra lỗi thì vẫn tiếp tục crawl k bị dừng lại
  try:
    print(i,mainURL+link)
    soup=getSoup(mainURL+link)
    row=[]
    # lấy ra tên sản phẩm
    row.append(soup.h1.text)
    # trích suất bảng thông số
    specs=soup.find('ul', class_='parameter__list')
    lis=specs.findAll('li')
    for li in lis[:9]:
      content=""
      for span in li.span:
        content+=span.string
      row.append(content)
    #lấy ra giá
    row.append(soup.find('p',class_='box-price-present').string)
    tgdd.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue


0 https://www.thegioididong.com/dtdd/samsung-galaxy-s22-ultra?src=osp
1 https://www.thegioididong.com/dtdd/iphone-11?src=osp
2 https://www.thegioididong.com/dtdd/oppo-reno7-z?src=osp
3 https://www.thegioididong.com/dtdd/realme-c35?src=osp
4 https://www.thegioididong.com/dtdd/xiaomi-11t?src=osp
5 https://www.thegioididong.com/dtdd/samsung-galaxy-a03?src=osp
6 https://www.thegioididong.com/dtdd/xiaomi-redmi-note-11-pro-4g?src=osp
7 https://www.thegioididong.com/dtdd/xiaomi-redmi-note-11s-5g?src=osp
8 https://www.thegioididong.com/dtdd/iphone-13-pro-max?src=osp
9 https://www.thegioididong.com/dtdd/samsung-galaxy-a52s-5g?src=osp
10 https://www.thegioididong.com/dtdd/vivo-y15s-2021?src=osp
11 https://www.thegioididong.com/dtdd/oppo-a55-4g?src=osp
12 https://www.thegioididong.com/dtdd/samsung-galaxy-s22-ultra-5g-256gb-dac-biet?src=osp
error:  12 'NoneType' object has no attribute 'text'
13 https://www.thegioididong.com/dtdd/samsung-galaxy-z-fold-3?src=osp
14 https://www.thegioididong.com/dtd

In [10]:
print("success: ",len(tgdd),",error: ",len(allLinks)-len(tgdd))

success:  92 ,error:  5


In [12]:
# chuyển thành dataframe
df = pd.DataFrame(tgdd, columns =["Name","Screen","OS","Rear camera","Front camera","Chip","RAM","ROM","SIM","Batery","Price"]) 
df.head()

,Name,Screen,OS,Rear camera,Front camera,Chip,RAM,ROM,SIM,Batery,Price
0,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,Dynamic AMOLED 2X,Android 12,"Chính 108 MP & Phụ 12 MP, 10 MP, 10 MP",40 MP,Snapdragon 8 Gen 1 8 nhân,8 GB,128 GB,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,5000 mAh,27.990.000₫
1,Điện thoại iPhone 11 64GB,IPS LCD,iOS 15,2 camera 12 MP,12 MP,Apple A13 Bionic,4 GB,64 GB,1 Nano SIM & 1 eSIM,3110 mAh,13.290.000₫ *
2,Điện thoại OPPO Reno7 Z 5G,AMOLED,Android 11,"Chính 64 MP & Phụ 2 MP, 2 MP",16 MP,Snapdragon 695 5G 8 nhân,8 GB,128 GB,2 Nano SIM (SIM 2 chung khe thẻ nhớ),4500 mAh,10.490.000₫
3,Điện thoại Realme C35 64GB,IPS LCD,Android 11,"Chính 50 MP & Phụ 2 MP, 0.3 MP",8 MP,Unisoc T616 8 nhân,4 GB,64 GB,2 Nano SIM,5000 mAh,4.290.000₫
4,Điện thoại Xiaomi 11T 5G 128GB,AMOLED,Android 11,"Chính 108 MP & Phụ 8 MP, 5 MP",16 MP,MediaTek Dimensity 1200,8 GB,128 GB,2 Nano SIM,5000 mAh,10.990.000₫


In [13]:
# ghi ra file csv
df.to_csv("thegioididong.csv")

## cellphones.com.vn 
Ngày crawl: 22/6/2022

In [14]:
def getLinkFromJSONCPS(url):
  """Hàm gọi api của cellphones và lấy ra link các sản phẩm"""
  # Sử dụng headers của postman để k bị chặn
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  listproducts=page.json()
  # chỉ lấy sản phẩm có giá
  links=[product['url'] for product in listproducts if float(product['final_price']) >0]
  return links


In [19]:
links=[]
# lấy ra link các sản phẩm từ trang 1 đến trang 25
for i in range(25):
  url="https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page={}&id=3&order=view_count2&dir=desc".format(i)
  print(url)
  links+=getLinkFromJSONCPS(url)


https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=0&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=1&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=2&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=3&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=4&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=5&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=6&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=7&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=8&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=9&id=3&order=view_count2&dir=desc
https://cellphones.com.vn/lapi

In [20]:
print("Number of link: ",len(links))

Number of link:  272


In [21]:
# dict để mapping các thông số theo đúng thứ tự
mappingCPS={
    'Kích thước màn hình': 0,
    'Công nghệ màn hình': 1,
    'Camera sau': 2,
    'Camera trước': 3,
    'Chipset': 4,
    'Dung lượng RAM': 5,
    'Bộ nhớ trong': 6,
    'Pin': 7,
    'Thẻ SIM': 8,
    'Hệ điều hành': 9,
    'Độ phân giải màn hình': 10,
    'Tính năng màn hình': 11,
    'Loại CPU': 12,
    'Trọng lượng': 13,
    'Bluetooth': 14,
    'Tần số quét': 15,
    }


In [22]:
cellphoneS=[]
for (i,link) in enumerate(links):
  # try catch để xảy ra lỗi thì không bị dừng lại
  try:
    print(i,link)
    row=[None for i in range(16)]
    soup=getSoup(link)
    # lấy ra bảng thông số
    table=soup.find('table',id="tskt")
    trs=table.findAll('tr')
    for tr in trs:
      ths=tr.findAll('th')
      row[mappingCPS[ths[0].text.strip()]]=ths[1].text.strip()
    # lấy ra tên sản phẩm
    row.insert(0,soup.find('h1').text.strip())
    # lấy ra giá
    row.append(soup.find('p',class_='special-price').text)
    cellphoneS.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue

0 https://cellphones.com.vn/iphone-11.html
1 https://cellphones.com.vn/iphone-13-pro-max.html
2 https://cellphones.com.vn/iphone-12-pro-max.html
3 https://cellphones.com.vn/samsung-galaxy-s22-ultra.html
4 https://cellphones.com.vn/iphone-12.html
5 https://cellphones.com.vn/xiaomi-redmi-note-11.html
6 https://cellphones.com.vn/iphone-13.html
7 https://cellphones.com.vn/xiaomi-redmi-note-11-pro-plus.html
8 https://cellphones.com.vn/iphone-11-128gb.html
9 https://cellphones.com.vn/xiaomi-redmi-note-11-128gb.html
10 https://cellphones.com.vn/samsung-galaxy-note-20-ultra-5g.html
11 https://cellphones.com.vn/xiaomi-redmi-note-11-pro.html
12 https://cellphones.com.vn/samsung-galaxy-s22-plus.html
13 https://cellphones.com.vn/oppo-a16k.html
14 https://cellphones.com.vn/nubia-red-magic-7.html
15 https://cellphones.com.vn/samsung-galaxy-a73.html
16 https://cellphones.com.vn/samsung-galaxy-m52.html
17 https://cellphones.com.vn/xiaomi-mi-11t.html
18 https://cellphones.com.vn/oppo-reno-6z.html
19 ht

In [23]:
print("success: ",len(cellphoneS),",error: ",len(links)-len(cellphoneS))

success:  263 ,error:  9


In [24]:
# chuyển thành dataframe
df = pd.DataFrame(cellphoneS, columns =["Name","Screen size","Screen type","Rear camera","Front camera","Chip","RAM","ROM","Batery","SIM","OS","Screen resolution","Screen feature","CPU type","Weight","Bluetooth","Refesh rate","Price"]) 
df.head()


,Name,Screen size,Screen type,Rear camera,Front camera,Chip,RAM,ROM,Batery,SIM,OS,Screen resolution,Screen feature,CPU type,Weight,Bluetooth,Refesh rate,Price
0,iPhone 11 64GB I Chính hãng VN/A,6.1 inches,IPS LCD,Camera kép 12MP:- Camera góc rộng: ƒ/1.8 apert...,"12 MP, ƒ/2.2 aperture",A13 Bionic,4 GB,64 GB,3110 mAh,Nano-SIM + eSIM,iOS 13 hoặc cao hơn (Tùy vào phiên bản phát hành),1792 x 828 pixel,True-tone,Hexa-core,194 g,5.0,60Hz,11.190.000 ₫
1,iPhone 13 Pro Max 128GB | Chính hãng VN/A,6.7 inches,OLED,"Camera góc rộng: 12MP, ƒ/1.5 Camera góc siêu ...","12MP, ƒ/2.2",Apple A15,6 GB,128 GB,"4,325mAh",2 SIM (nano‑SIM và eSIM),iOS15,2778 x 1284 pixel,"120Hz, Super Retina XDR với ProMotion 6.1‑inch...",None,240g,v5.0,120Hz,27.590.000 ₫
2,iPhone 12 Pro Max 128GB I Chính hãng VN/A,6.7 inches,OLED,"Camera chính: 12 MP, f/1.6, 26mm, 1.4µm, dual ...","12 MP, f/2.2, 23mm (wide), 1/3.6"" SL 3D, (dept...",Apple A14 Bionic (5 nm),6 GB,128 GB,"Li-Ion, sạc nhanh 20W, sạc không dây 15W, USB ...",2 SIM (nano‑SIM và eSIM),iOS 14.1 hoặc cao hơn (Tùy vào phiên bản phát ...,1284 x 2778 pixels,HDR10Dolby VisionTrue-toneĐộ sáng 800 nits,Hexa-core,228,"5.0, A2DP, LE",60Hz,26.590.000 ₫
3,Samsung Galaxy S22 Ultra (8GB - 128GB),6.8 inches,Dynamic AMOLED 2X,"108 MP, f/1.8 góc rộng10 MP, f/4.910 MP, f/2.4...","40 MP, f/2.2",Qualcomm Snapdragon 8 Gen 1 (4 nm),8 GB,128 GB,Li-Ion 5000 mAh,2 Nano SIM hoặc 1 Nano + 1 eSIM,"Android 12, One UI 4.1",1440 x 3088 pixels (QHD+),Tần số quét 120HzCông nghệ HDR10+,Octa-core (1x3.00 GHz Cortex-X2 & 3x2.50 GHz C...,229 g,"5.2, A2DP, LE",None,26.190.000 ₫
4,iPhone 12 64GB I Chính hãng VN/A,6.1 inches,OLED,"12 MP, f/1.6, 26mm (wide), 1.4µm, dual pixel P...","12 MP, f/2.2, 23mm (wide), 1/3.6"" SL 3D, (dept...",Apple A14 Bionic (5 nm),4 GB,64 GB,"Li-Ion, sạc nhanh 20W, sạc không dây 15W, USB ...",2 SIM (nano‑SIM và eSIM),iOS 14.1 hoặc cao hơn (Tùy vào phiên bản phát ...,1170 x 2532 pixels,HDR10Dolby VisionTrue-tone,Hexa-core,164 g,"5.0, A2DP, LE",60Hz,15.890.000 ₫


In [25]:
# ghi vào file csv
df.to_csv("cellphoneS.csv")

## hoanghamobile.com
Ngày crawl: 22/6/2022

In [ ]:
mainURL="https://hoanghamobile.com"
url="https://hoanghamobile.com/dien-thoai-di-dong?p=8"
soup=getSoup(url)
links=soup.findAll('div',class_="promote")
# lấy ra tất cả link sản phẩm tại trang p=8
links=[mainURL+link.a.get('href') for link in links]

In [ ]:
mappingHH={
      'Công nghệ màn hình::':0,
      'Độ phân giải::':1,
      'Màn hình rộng::':2,
      'Độ phân giải:':3,
      'Hệ điều hành:':4,
      'Chip xử lý (CPU):':5,
      'Bộ nhớ trong (ROM):':6,
      'RAM:':7,
      'Mạng di động:':8,
      'Số khe sim:':9,
      'Dung lượng pin:':10,
}

In [ ]:
# file 1 từ 0->346 thì server bị lỗi , chạy lần 2 từ 347 đến hết
hoangHa=[]
for (i,link) in enumerate(links[347:]):
  # try catch để khi lỗi sẽ không dừng lại
  try:
    print(i+347,link)
    soup=getSoup(link)
    specs={s.strong.text.strip():s.span.text.strip() for s in soup.find('div','specs-special').findAll('li') }
    row=[None for i in range(11)]
    for (key,val) in specs.items():
      row[mappingHH[key]]=val
    # lấy ra tên sản phẩm
    name=soup.find('div','top-product').find('h1').text.strip()
    row.insert(0,name)
    # lấy ra giá sản phẩm
    price=soup.find('p',class_='current-product-price').strong.text.strip()
    row.append(price) 
    hoangHa.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue


347 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-13-pro-max-512gb-chinh-hang-vn-a
348 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-13-pro-max-1TB-chinh-hang-vn
349 https://hoanghamobile.com/dien-thoai-di-dong/iphone-11-64gb-chinh-hang-vn-a
350 https://hoanghamobile.com/dien-thoai-di-dong/iphone-11-128gb-chinh-hang-vn-a
351 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-128gb-chinh-hang-vn-a
352 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-256gb-chinh-hang-vn-a
353 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-64gb-chinh-hang-vn-a
354 https://hoanghamobile.com/dien-thoai-di-dong/oppo-a55-chinh-hang
355 https://hoanghamobile.com/dien-thoai-di-dong/oppo-a95-chinh-hang
356 https://hoanghamobile.com/dien-thoai-di-dong/oppo-a16-3g-32g-chinh-hang
357 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-mini-64gb-chinh-hang-vn-a
358 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-mini-128gb-chinh-hang-v

File hoangHa1.csv từ dòng 1->346

File hoangHa2.csv từ dòng 347-> 1263

In [ ]:
# chuyển thành dataframe
df = pd.DataFrame(hoangHa, columns =["Name","Screen type","Screen resolution","Screen size","Camera","OS","Chip","ROM","RAM","Mobile network","SIM","Batery","Price"]) 
df.to_csv("hoangHa2.csv")


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               911 non-null    object
 1   Screen type        875 non-null    object
 2   Screen resolution  785 non-null    object
 3   Screen size        442 non-null    object
 4   Camera             779 non-null    object
 5   OS                 732 non-null    object
 6   Chip               719 non-null    object
 7   ROM                785 non-null    object
 8   RAM                803 non-null    object
 9   Mobile network     680 non-null    object
 10  SIM                719 non-null    object
 11  Batery             710 non-null    object
 12  Price              911 non-null    object
dtypes: object(13)
memory usage: 92.6+ KB


## nguyenkim.com

Ngày crawl: 17/6/2022

In [26]:
# lấy ra link các sản phẩm từ trang 1 đến trang 8
linksNK=[]
for i in range(1,8):
  soup=getSoup("https://www.nguyenkim.com/dien-thoai-di-dong/page-{}/".format(i))
  links=soup.findAll('div',class_="product-title")
  links=[link.a.get('href') for link in links]
  linksNK=linksNK+links

linksNK

['https://www.nguyenkim.com/dien-thoai-iphone-12-64gb-xanh-la.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-12-128gb-do.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xam.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xanh.html',
 'https://www.nguyenkim.com/iphone-13-pro-128gb-bac.html',
 'https://www.nguyenkim.com/iphone-13-pro-256gb-xam.html',
 'https://www.nguyenkim.com/iphone-13-pro-256gb-xanh-duong.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-13-pro-max-128gb-vang-dong-vi.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-11-64gb-trang-vi-2.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-11-64gb-tim-vi-2.html',
 'https://www.nguyenkim.com/dien-thoai-xiaomi-redmi-9a-2gb-32gb-xanh-duong.html',
 'https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a52-4g-8gb-128gb-xanh.html',
 'https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a32-4g-6gb-128gb-xanh.html',
 'https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a52-4g-8gb-128

In [27]:
print("Number of link:",len(linksNK))

Number of link: 214


In [28]:
mappingNK={
      'Loại màn hình:':0,
      'Kích thước màn hình:':1,
      "Độ phân giải màn hình:": 2,
      'Camera trước:':3,
      'Camera sau:':4,
      'Hệ điều hành:':5,
      'Chipset:':6,
      'Bộ nhớ trong:':7,
      'RAM:':8,
      'Dung lượng Pin:':9,
}

In [29]:
nguyenKim=[]
for (i,link) in enumerate(linksNK):
  # try catch để xảy ra lỗi vẫn không dừng lại
  try:
    print(i,link)
    row=[None for i in range(10)]
    soup=getSoup(link)
    # lấy ra bảng thông số
    table=soup.find('tbody',class_="popup")
    trs=table.findAll('tr')
    for tr in trs:
      tds=tr.findAll('td')
      if tds[0].text.strip() in mappingNK:
        row[mappingNK[tds[0].text.strip()]]=tds[1].text.strip()
    # lấy ra tên sản phẩm
    row.insert(0,soup.find('h1',class_="product_info_name").text.strip())
    # lấy ra giá sản phẩm
    row.append(soup.find('span',class_="nk-price-final").text.strip())
    nguyenKim.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue

0 https://www.nguyenkim.com/dien-thoai-iphone-12-64gb-xanh-la.html
1 https://www.nguyenkim.com/dien-thoai-iphone-12-128gb-do.html
2 https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xam.html
3 https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xanh.html
4 https://www.nguyenkim.com/iphone-13-pro-128gb-bac.html
5 https://www.nguyenkim.com/iphone-13-pro-256gb-xam.html
6 https://www.nguyenkim.com/iphone-13-pro-256gb-xanh-duong.html
7 https://www.nguyenkim.com/dien-thoai-iphone-13-pro-max-128gb-vang-dong-vi.html
8 https://www.nguyenkim.com/dien-thoai-iphone-11-64gb-trang-vi-2.html
9 https://www.nguyenkim.com/dien-thoai-iphone-11-64gb-tim-vi-2.html
10 https://www.nguyenkim.com/dien-thoai-xiaomi-redmi-9a-2gb-32gb-xanh-duong.html
11 https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a52-4g-8gb-128gb-xanh.html
12 https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a32-4g-6gb-128gb-xanh.html
13 https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a52-4g-8gb-128gb-den.html
14 https:/

In [32]:
print("success: ",len(nguyenKim),",error: ",len(linksNK)-len(nguyenKim))

success:  214 ,error:  0


In [33]:
# chuyển thành dataframe
df = pd.DataFrame(nguyenKim, columns =["Name","Screen type","Screen size","Screen resolution","Front camera","Rear camera","OS","Chip","ROM","RAM","Batery","Price"]) 
df

,Name,Screen type,Screen size,Screen resolution,Front camera,Rear camera,OS,Chip,ROM,RAM,Batery,Price
0,Điện thoại iPhone 12 64GB Xanh lá,Super Retina XDR,"6.1 inch """,2532 x 1170 pixels,12MP,12 MP (kép),iOS 14.1,Apple A14 Bionic,64 GB,4 GB,None,16.090.000đ
1,Điện thoại iPhone 12 128GB Đỏ,Super Retina XDR,"6.1 inch """,2532 x 1170 pixels,12 MP,12 MP (kép),iOS 14.1,Apple A14 Bionic,128 GB,4 GB,None,17.990.000đ
2,Điện thoại iPhone 12 Pro 256GB Xám,None,"6.1 """,None,12 MP,"12 MP, 12 MP, 12 MP",iOS 14.1,Apple A14 Bionic,256 GB,6 GB,None,24.990.000đ
3,Điện thoại iPhone 12 Pro 256GB Xanh,None,"6.1 """,None,12 MP,"12 MP, 12 MP, 12 MP",iOS 14.1,Apple A14 Bionic,256 GB,6 GB,None,28.990.000đ
4,Điện thoại iPhone 13 Pro 128GB Bạc,OLED,"6.1 """,2532 x 1170 pixels,12MP,12MP-12MP-12MP,iOS 15,Apple A15 Bionic,128GB,6GB,None,25.990.000đ
...,...,...,...,...,...,...,...,...,...,...,...,...
209,Điện thoại iPhone 12 64GB Xanh lá,Super Retina XDR,"6.1 inch """,2532 x 1170 pixels,12MP,12 MP (kép),iOS 14.1,Apple A14 Bionic,64 GB,4 GB,None,16.090.000đ
210,Samsung Galaxy S20+ Đen,Dynamic AMOLED 2x,"6.7 """,None,10 MP,4 MP + 12 MP+ 12 MP,"Android 10, One UI 2.1",Exynos 990,128 GB,8 GB,4500 mAh,23.990.000đ
211,Điện thoại Samsung Galaxy S20 Ultra Xám,Dynamic AMOLED 2x,"6.9 """,None,40 MP,108 MP + 48 MP + 12 MP,"Android 10, One UI 2.1",Exynos 990,128 GB,12 GB,5000 mAh,29.990.000đ
212,Điện thoại Samsung Galaxy A52 4G 8GB/128GB Tím,Màn hình tràn viền vô cực Super AMOLED Full HD...,"6.5 """,None,32MP,4 camera: 64MP+12MP + 5MP + 5MP,None,Vi xử lý 8 nhân Snapdragon 720G,128GB,8GB,"4500mAh, Công nghệ Sạc nhanh siêu tốc 25W",9.290.000đ


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               214 non-null    object
 1   Screen type        181 non-null    object
 2   Screen size        213 non-null    object
 3   Screen resolution  147 non-null    object
 4   Front camera       205 non-null    object
 5   Rear camera        204 non-null    object
 6   OS                 167 non-null    object
 7   Chip               200 non-null    object
 8   ROM                214 non-null    object
 9   RAM                211 non-null    object
 10  Batery             113 non-null    object
 11  Price              214 non-null    object
dtypes: object(12)
memory usage: 20.2+ KB


In [ ]:
# lưu vào file csv
df.to_csv("nguyenkim.csv")